In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from xgboost import XGBRegressor

# Load dataset
df = pd.read_csv("timing_performance_dataset.csv")

# Identify the correct target column
print("Dataset Columns:", df.columns)  # Print column names to identify target
target_column = "Timing_Violation_Probability"  # Replace with the correct target column

if target_column not in df.columns:
    raise KeyError(f"Column '{target_column}' not found in dataset. Available columns: {df.columns}")

# Handle missing values
df.fillna(df.mean(), inplace=True)

X = df.drop(columns=[target_column])
y = df[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the Neural Network model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

# Alternative Model: XGBoost
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train)

# Evaluate XGBoost model
xgb_predictions = xgb_model.predict(X_test)
xgb_mae = np.mean(np.abs(y_test - xgb_predictions))
print(f"XGBoost MAE: {xgb_mae}")


Dataset Columns: Index(['Setup_Time_Violations', 'Hold_Time_Violations', 'Critical_Path_Delay',
       'Propagation_Delay', 'Clock_Skew', 'Clock_Domain_Crossings',
       'Fan_In_Count', 'Fan_Out_Count', 'Combinational_Logic_Levels',
       'Pipeline_Stages', 'Register_to_Register_Path_Length',
       'Multiplexer_Depth', 'Gate_Count', 'Logic_Optimization_Ratio',
       'Resource_Utilization', 'Retiming_Factor', 'Clock_Gating_Ratio',
       'Combinational_Depth', 'Timing_Violation_Probability'],
      dtype='object')
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 1.4772 - mae: 0.7187 - val_loss: 0.9423 - val_mae: 0.3881 - learning_rate: 5.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1012 - mae: 0.5030 - val_loss: 0.8547 - val_mae: 0.3372 - learning_rate: 5.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9917 - mae: 0.4431 - val_loss: 0.8053 - val_mae: 0.3116 - learning_rate: 5.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms